GoogleSheet for planning and documentation: 
* https://docs.google.com/spreadsheets/d/1fI7d_93jsmje8tEvsGzLxQ5aRxOWq-e0KeXekkhvuXM/

In [ ]:
#Import goes here
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import random
from datetime import date
#import sqldf

In [ ]:
#Read in data from Github
board = pd.read_csv("https://raw.githubusercontent.com/hangxu0324/Capstone-Project/main/data/board_anon.csv",index_col=0)
cx_df = pd.read_csv("https://raw.githubusercontent.com/hangxu0324/Capstone-Project/main/data/cx_df_anon.csv",index_col=0)
gx_df = pd.read_csv("https://raw.githubusercontent.com/hangxu0324/Capstone-Project/main/data/gx_df_anon.csv",index_col=0)
launch_df = pd.read_csv("https://raw.githubusercontent.com/hangxu0324/Capstone-Project/main/data/launch_df_anon.csv")
#New provided data
#df_candidate = pd.read_csv("https://raw.githubusercontent.com/hangxu0324/Capstone-Project/main/data/material_candidates/df_candidates.csv")
country_pop = pd.read_csv("https://raw.githubusercontent.com/hangxu0324/Capstone-Project/main/data/material_candidates/population_anon.csv")
reg_deg = pd.read_csv("https://raw.githubusercontent.com/hangxu0324/Capstone-Project/main/data/material_candidates/reg_desig_anon.csv")
indication_df = pd.read_csv("https://raw.githubusercontent.com/hangxu0324/Capstone-Project/main/data/material_candidates/indication_df_anon.csv")

In [ ]:
print(len(board.cluster_id.unique()))
#print(len(df_candidate.cluster_id.unique()))
#print(len(df_candidate_new.cluster_id.unique()))

1464


## Preprocessing functions

Function `data_filter`: 
1. Filter out clusters that have a growth stage.
2. Filter out clusters that have a launch data after 2006.
3. If include_all=True, the filter will include data from all stages. Else, only data points that are either at the stage of growth / transitioned from growth will be included.
4. All data after 2019/4/1 (testing data) will be included.

Function `process_launch_date`: Fix some incorrect values in the launch dataset

1. If the launch date is null, set it to the start of the cluster.
2. If the launch date is after the start date of the cluster, set it to the start of the cluster.

In [ ]:
# data filter function
def data_filter(board, time_filter=False, include_all=False):

  ''' 
  input: board is the dataframe from df_candidate_anon.csv
  output: growth_board with 706 clusters that contain growth stages and have launch date after 2006
                         Or 847 clusters that contain growth stages

  If include_all set to false, data points that are either at the stage of growth / Maturity (Growth before) / Unknown will be included.                       
  '''
  board.loc[board['stage_name'] =='Forecasting', 'stage_name'] = np.nan
  ## find all clusters with growth
  growth_ids = board[board['stage_name']=='Growth'].cluster_id.unique()

  ## choose clusters with launch date after 2006
  if time_filter: 
    # find growth clusters with launch date
    growth_launch_df = clean_launch_df[(clean_launch_df['cluster_id'].isin(growth_ids))].reset_index()
    # apply filter >2006
    growth_launch_df_filtered = growth_launch_df[pd.DatetimeIndex(growth_launch_df['launch_date']).year > 2006]
    growth_ids = growth_launch_df_filtered.cluster_id.unique()

  growth_board = board[board['cluster_id'].isin(growth_ids)].reset_index()

  if not include_all:
      ## Only keep three scenarios: stage_name = 'Growth' or stage_name = 'Maturity' and lag = 'Growth' or stage_name = Forecasting
      growth_board = growth_board[(growth_board['stage_name']=='Growth') | \
                       ((growth_board['stage_name'] == 'Maturity') & (growth_board['stage_name_lag_1'] == 'Growth')) | \
                       (growth_board['stage_name'].isnull())].reset_index(drop=True)
  
  return growth_board

In [ ]:
# a=data_filter(board)
# a[a['cluster_id']=='ID_18']

In [ ]:
def process_launch_date(launch_df):

  ''' 
  input: launch_df is the dataframe from launch_df.csv
  output: launch_df with adjusted launch date
  '''
  start_date = board.sort_values('date').groupby('cluster_id').head(1)[['cluster_id','date']]
  start_date = start_date.rename(columns={'date': 'start_date'})

  ## when launch date is null, set it to start date of the cluster
  temp_df = pd.merge(launch_df,start_date,on='cluster_id',how='outer')[['cluster_id','launch_date','start_date']]
  temp_df['launch_date'] = temp_df['launch_date'].fillna(temp_df['start_date'])

  ## when launch date is after start date of the cluster, set it to start date of the cluster
  temp_df = temp_df.dropna().reset_index()
  for i in range(len(temp_df)):
    if temp_df.loc[i,'launch_date'] > temp_df.loc[i,'start_date']:
      temp_df.loc[i,'launch_date'] = temp_df.loc[i,'start_date']
      
  return temp_df

## Feature Engineering functions

### Senqi 
Create the following features:

1. launch months: how many months since the launch date.

2. time_since_lost_exclusivity: the number of months since the time this drug lost its exclusivity (if this drug never has exclusivity, return NaN).

3. transition_point: if the transition point from growth to maturity happned (0 if not transition point, 1 if transition).

4. transition point augmentation: augment points surrounding the transition point by the given period.




In [ ]:
def launch_months(center_df):

    center_df['launch_months'] = center_df.apply(lambda x: (x.date.year - x.launch_date.year) * 12 + (x.date.month - x.launch_date.month), axis=1)

    return center_df

def time_since_no_exclusivity(center_df):
    center_df['time_since_no_exclusivity'] = center_df.apply(lambda x: (x.date.year - x.gx_entry_date.year) * 12 + 
                                                             (x.date.month - x.gx_entry_date.month), axis=1)

    return center_df


def transition_point(center_df, all_transistion = False):
    '''
    If all_transition set to True, all transition point will be marked as 1.
    Otherwise, only growth-maturity transition will be marked as 1.
    '''

    center_df['transition_point'] = center_df.groupby(['cluster_id','stage_name','stage_name_lag_1'])['date'].rank(method='first')
    transition = pd.DataFrame(center_df[center_df.transition_point == 1]['transition_point'])
    center_df['transition_point'] = transition
    center_df['transition_point'] = center_df['transition_point'].replace(np.nan, 0)
        
    if all_transistion:   
        return center_df
    else:
        center_df['transition_point'] = center_df.apply(lambda x: 0 if x.stage_name != 'Maturity' else x.transition_point,axis=1)
        return center_df

def label_augment(center_df, aug_period):
    '''
    Perform data augmentation with the given period n.
    For any point that has a label of 1, previous and next n rows will be augmented as 1.
    '''
    l = len(center_df)
    copy = center_df.copy()

    for i in range(l):
        current_row = copy.loc[i,:]
     
        try:
          if current_row.transition_point == 1:
                for j in range(1,aug_period+1):
                    prev_row = copy.loc[i-j,:]           
                    next_row = copy.loc[i+j,:]

                    if next_row.cluster_id == current_row.cluster_id:                
                        center_df.at[i+j,'transition_point'] = 1

                    if prev_row.cluster_id == current_row.cluster_id:
                        center_df.at[i-j,'transition_point'] = 1                   
        except:
            continue

    return center_df

def categoricalize(center_df):
    center_df.country = pd.Categorical(center_df.country)
    center_df.cluster = pd.Categorical(center_df.cluster)
    center_df.cluster_id = pd.Categorical(center_df.cluster_id)
    center_df.business_unit = pd.Categorical(center_df.business_unit)
    center_df.ther_area_fact = pd.Categorical(center_df.ther_area_fact)

    return center_df

def senqi_transform(center_df,aug_period=4):
    center_df = launch_months(center_df)
    center_df = time_since_no_exclusivity(center_df)
    center_df = transition_point(center_df)
    center_df = label_augment(center_df,aug_period)

    return center_df

###  Shuyue 

1. if_exclusive		binary variable indicating if at this date the drug has exclusivity (gx data)
2. volume_daily_avg		calculate the daily average volume by dividing monthly number to the days in that month
3. volume_lag_1m_pctg		last month's daily volume/current daily volume
4. volume_lag_2m_pctg		second last month's daily volume/current daily volume
5. volume_lag_3m_pctg		third last month's daily volume/current daily volume


In [ ]:
### Volume related features might not be available
### 0 means not exclusive, 1 means exclusive 
def volume_daily_avgcenter_df(center_df):
  center_df["volume_daily_avg"] = center_df['volume']/center_df['date'].dt.days_in_month
  return center_df;
def volume_lag_1m_pctg(center_df):
  center_df["volume_lag_1m_pctg"] = center_df.groupby(['cluster_id']).shift(1)['volume_daily_avg']/center_df['volume_daily_avg']
  return center_df;
def volume_lag_2m_pctg(center_df):
  center_df["volume_lag_2m_pctg"] = center_df.groupby(['cluster_id']).shift(2)['volume_daily_avg']/center_df['volume_daily_avg']
  return center_df;
def volume_lag_3m_pctg(center_df):
  center_df["volume_lag_3m_pctg"] = center_df.groupby(['cluster_id']).shift(3)['volume_daily_avg']/center_df['volume_daily_avg']
  return center_df;

def shuyue_transform(center_df):
  center_df = volume_daily_avgcenter_df(center_df)
  center_df = volume_lag_1m_pctg(center_df)
  center_df = volume_lag_2m_pctg(center_df)
  center_df = volume_lag_3m_pctg(center_df)

  return center_df

### Hang
1. vol_norm: nomralized volume
2. vol_std_3m: std. of vol_nomr in the past 3 months
3. vol_std_6m: std. of vol_nomr in the past 6 months
4. vol_std_diff: vol_std_3m - vol_std_6m
5. population
6. reg_deg

In [ ]:
def feature_hang(board, country_pop = country_pop, reg_deg = reg_deg, indication_df = indication_df):
  # temp = board
  # # volume change rate
  # temp = temp.sort_values(by=['country','cluster_id', 'date'], ascending= True)
  # temp['volume_lag1'] = temp.groupby(['country','cluster_id'])['volume'].shift(1)
  # temp['volume_change_r'] = np.log(temp['volume_lag1'] / temp['volume'])
  
  # # vol_norm
  # temp_agg = temp.groupby(['country','cluster_id'])['volume_change_r'].agg(['mean','std'])
  # new = temp.join(temp_agg, on = ['country','cluster_id'], how='left', rsuffix='_agg')
  # new = new[['country','cluster_id','date','volume_change_r','mean','std']]
  # new['vol_norm'] = (new['volume_change_r'] - new['mean']) / new['std']

  # # vol_std_3m, vol_std_6m, vol_std_diff
  # new = new.sort_values(by=['country','cluster_id', 'date'], ascending= True)
  # new['vol_lag1'] = new.groupby(['country','cluster_id'])['vol_norm'].shift(1)
  # new['vol_lag2'] = new.groupby(['country','cluster_id'])['vol_norm'].shift(2)
  # new['vol_lag3'] = new.groupby(['country','cluster_id'])['vol_norm'].shift(3)
  # new['vol_lag4'] = new.groupby(['country','cluster_id'])['vol_norm'].shift(4)
  # new['vol_lag5'] = new.groupby(['country','cluster_id'])['vol_norm'].shift(5)
  # new['vol_lag6'] = new.groupby(['country','cluster_id'])['vol_norm'].shift(6)
  # new['vol_std_3m'] = new.loc[:,['vol_lag1','vol_lag2','vol_lag3']].std(axis = 1, skipna = False)
  # new['vol_std_6m'] = new.loc[:,['vol_lag1','vol_lag2','vol_lag3','vol_lag4','vol_lag5','vol_lag6']].std(axis = 1, skipna = False)
  # new['vol_std_diff'] = new['vol_std_3m'] - new['vol_std_6m']

  # output = new.loc[:, ['country','cluster_id','date',\
  #                     'vol_norm','vol_std_3m','vol_std_6m','vol_std_diff']]
  # output = board.merge(output, on = ['country','cluster_id', 'date'], how='left')
  output = board.copy()
  # population
  output['year'] = pd.DatetimeIndex(output.date).year
  output = output.merge(country_pop.drop(['Unnamed: 0'], axis=1), left_on = ['country', 'year'], right_on =  ['country', 'year'], how = 'left')

  # reg_deg
  output = output.merge(reg_deg.drop(['Unnamed: 0'], axis=1), left_on = 'cluster_id', right_on = 'cluster_id', how = 'left')

  # indication_df
  indication_df['date'] = pd.to_datetime(indication_df['date'])
  temp = indication_df.drop(['Unnamed: 0'], axis = 1).groupby(['cluster_id', 'date']).indication_entry.sum().reset_index(name ='indication_num')
  output = output.merge(temp, left_on = ['cluster_id','date'], right_on = ['cluster_id','date'], how = 'left')
  output['indication_num'] = output['indication_num'].fillna(0)
  output['indication_num'] = output.sort_values(['cluster_id','date']).groupby(['cluster_id'])['indication_num'].cumsum()

  return output

In [ ]:
# may cause information leak when doing validation in the modeling

def feature_hang_addition(board):
  output = board.copy()
  # all time mean and standard deviation of the cluster_id
  output_right = output.groupby(['cluster_id']).agg(vol_mean_alltime = ('volume', np.mean), vol_std_alltime = ('volume', np.std)).reset_index()
  output = output.merge(output_right, how = 'left', on = 'cluster_id')
  output_right = output.query('date<="2016-03-01"').groupby(['cluster_id']).agg(vol_mean_before_2016 = ('volume', np.mean), vol_std_before_2016 = ('volume', np.std)).reset_index()
  output = output.merge(output_right, how = 'left', on = 'cluster_id')
  #
  output_right = output.query('date<="2016-03-01" & date >= "2015-11-01" ').groupby(['cluster_id']).agg(vol_mean_before_4m = ('volume', np.mean), vol_std_before_4m = ('volume', np.std)).reset_index()
  output = output.merge(output_right, how = 'left', on = 'cluster_id')
  #
  output_right = output.query('date<="2016-03-01" & date >= "2015-03-01" ').groupby(['cluster_id']).agg(vol_mean_before_1y = ('volume', np.mean), vol_std_before_1y = ('volume', np.std)).reset_index()
  output = output.merge(output_right, how = 'left', on = 'cluster_id')
  
  # month_since_growth
  temp = output.query(" stage_name == 'Growth' ").sort_values(['cluster_id','date']).loc[:,['date','cluster_id']]
  temp['rn'] = temp.groupby(['cluster_id']).rank(method = 'first', ascending = True)
  output_right = temp.query("rn == 1").rename(columns = {'date':'growth_date'})
  output = output.merge(output_right, on = 'cluster_id', how = 'left')
  output = output.assign(month_since_growth = lambda x: (x.date.dt.year -x.growth_date.dt.year) * 12 + (x.date.dt.month -x.growth_date.dt.month)).drop(columns = 'rn')
  
  # x1-x4
  # output = output.assign(
  #   x1 = lambda x: 100* x.vol_std_before_2016 / x.vol_mean_before_2016 ,
  #   x2 = lambda x: 100* (x.vol_std_before_4m - x.vol_std_before_1y) / x.vol_std_before_1y ,
  #   x3 = lambda x: 100* (x.vol_mean_before_4m - x.vol_mean_before_1y) / x.vol_mean_before_1y,
  #   x4 = lambda x: 100* (x.vol_std_before_4m - x.vol_std_before_2016) / x.vol_std_before_2016,
  #   )
  # output.x1 = output.x1.fillna(0)
  # output.x2 = output.x2.fillna(0)
  # output.x3 = output.x3.fillna(0)
  # output.x4 = output.x4.fillna(0)
  return output

There are clusters without growth stage. We need to look at it closer. 

### Lindsey

comp_num: count the current number of competitors in the market

first_comp_month: calculate how many months since the first competitor enter the market (duration, 0 if no competitors, 1 for the first month of competitors entering)

if_exclusive: binary variable indicating if at this date the drug has exclusivity (gx data)

last_ind_days: calculate number of months since last indication was found 

last_stage_days: calculate number of days since last stage

volume_lag_1m_pctg:		last month's daily volume/current daily volume

volume_lag_2m_pctg:		second last month's daily volume/current daily volume

volume_lag_3m_pctg:		third last month's daily volume/current daily volume


In [ ]:
def comp_num(center_df, cx_df):
  def return_rolling_competitors(row):
    return len(cx_df[(cx_df.cluster_id == row.cluster_id) & (cx_df.competitor_entry_date <= row.date)])
  
  cx_df['competitor_entry_date'] = pd.to_datetime(cx_df['competitor_entry_date'])
  
  return center_df.apply(return_rolling_competitors, axis = 1)

def first_comp_month(center_df, cx_df):
  earliest_competitor = cx_df.groupby(['cluster_id']).agg({'competitor_entry_date': 'min'}).reset_index().rename({'competitor_entry_date': 'first_comp_month'}, axis = 1)
  center_df_temp = center_df.merge(earliest_competitor, on ='cluster_id', how = 'left')
  center_df_temp['first_comp_month'] = pd.to_datetime(center_df_temp['first_comp_month'])
  return (center_df_temp['date'] - center_df_temp['first_comp_month']).fillna(pd.Timedelta(-1, "d"))

def is_exclusive(center_df):
  def if_exclusive_row(row):
    if str(row.lose_exclusivity_date) == 'NaT':
      return True
    else:
      return row.date < row.lose_exclusivity_date
  generics_date = center_df.groupby(['cluster_id', 'date']).agg({'gx_entry_date': 'min'}).reset_index().rename({'gx_entry_date': 'lose_exclusivity_date'}, axis = 1)
  generics_date['lose_exclusivity_date'] = pd.to_datetime(generics_date['lose_exclusivity_date'])
  center_df_temp = center_df.merge(generics_date, on = ['date', 'cluster_id'])
  return center_df_temp.apply(if_exclusive_row, axis=1)

def last_stage_days(center_df):
  stage_entry_date = center_df.groupby(['cluster_id', 'stage_name']).agg({'date': 'min'}).reset_index().rename({'date': 'first_month_of_current_stage'}, axis = 1)
  center_df_temp = center_df.merge(stage_entry_date, on = ['cluster_id', 'stage_name'], how = 'left')
  return center_df_temp.date - center_df_temp.first_month_of_current_stage  

def last_ind_days(center_df, indication_df):
  indication_agg = indication_df.groupby(['cluster_id', 'date']).agg({'indication_entry': 'sum'}).reset_index().sort_values(['cluster_id','date'])
  # indication_agg[indication_agg.indication_entry > 0]

  indication_agg['indication_num'] = indication_agg.groupby(['cluster_id'])['indication_entry'].cumsum()
  indication_agg = indication_agg.groupby(['cluster_id', 'indication_num']).agg({'date': 'min'}).reset_index()
  indication_agg['date'] = pd.to_datetime(indication_agg.date)
  indication_agg = indication_agg.rename({'date': 'last_indication_date'}, axis=1)
  indication_agg['last_indication_date'] = indication_agg['last_indication_date']

  temp = center_df.merge(indication_agg, on = ['cluster_id', 'indication_num'], how = 'left')
  return (temp['date'] - temp['last_indication_date']).fillna(pd.Timedelta(-1, "d"))

def feature_lindsey(center_df):
  center_df['comp_num'] = comp_num(center_df, cx_df)
  center_df['first_comp_month'] = first_comp_month(center_df, cx_df).apply(lambda x:x.days)
  center_df['is_exclusive'] = is_exclusive(center_df)
  center_df['last_stage_days'] = last_stage_days(center_df).apply(lambda x:x.days)
  center_df['last_ind_days'] = last_ind_days(center_df, indication_df)
  # center_df['volume_lag_1m_pctg'] = volume_lag_1m_pctg(center_df)
  # center_df['volume_lag_2m_pctg'] = volume_lag_2m_pctg(center_df)
  # center_df['volume_lag_3m_pctg'] = volume_lag_3m_pctg(center_df)
  # center_df['relative_volume'] = center_df['volume']/center_df['population']
  return pd.concat([center_df, pd.get_dummies(center_df.date.apply(lambda x: x.month))], axis=1)
    

In [ ]:
# def impute_prevalence_pct(final_df, board):
#   for country in final_df.country.unique():
#     final_df.loc[(final_df.prevalence_pct<=0) & (final_df.country==country),'prevalence_pct'] = board[(board.prevalence_pct>=0) & (board.country==country)].prevalence_pct.median()
#   final_df.prevalence_pct.fillna(board[board.prevalence_pct>=0].prevalence_pct.median())

def impute_popuation(final_df, country_pop):
  for country in final_df.country.unique():
    final_df.loc[final_df.country==country,'population'].fillna(country_pop[country_pop.country==country].population.median(), inplace=True)
  final_df.population.fillna(country_pop.population.median(), inplace=True)

def volume_change_rate(final_df):
  temp_agg = final_df.groupby(['country','cluster_id'])['volume'].agg(['mean','std'])
  new = final_df.join(temp_agg, on = ['country','cluster_id'], how='left', rsuffix='_agg')
  new = new[['country','cluster_id','date','volume','mean','std']]
  new['vol_norm'] = (new['volume'] - new['mean']) / new['std']
  
  temp = new.sort_values(by=['country','cluster_id', 'date'], ascending= True)
  temp['volume_lag1'] = temp.groupby(['country','cluster_id'])['vol_norm'].shift(1)
  return np.log(temp['volume_lag1'] / temp['vol_norm'])

def feature_lindsey_addition(final_df):
  # impute_prevalence_pct(final_df, board)
  impute_popuation(final_df, country_pop)
  final_df = final_df.sort_values(by=['country','cluster_id', 'date'], ascending= True)
  final_df['prevalence'] = final_df['prevalence'] * final_df['population']
  final_df['volume_lag_1m_pctg'] = volume_change_rate(final_df)
  return final_df

## Feature Engineering Driver Codes

Let's include all preprocessing and transform in the following chunks

In [ ]:
print(board.shape)
## clean launch_df
clean_launch_df = process_launch_date(launch_df)
## filter data
center_df = data_filter(board,include_all=False)
print(center_df.shape)
print(center_df.cluster_id.unique().shape)

## Merge tables
## Merge with competitor dataset (Note this will increase data as relation is not 1-1)
#center_df = center_df.merge(cx_df.drop(['country'],axis=1),on='cluster_id',how='left')

## Merge exclusive data 
center_df = center_df.merge(gx_df.drop(['country'],axis=1),on='cluster_id',how='left')

## Merge launch date
center_df = center_df.merge(clean_launch_df,on='cluster_id',how='left')

## Convert to datetime
center_df['date'] = pd.to_datetime(center_df['date'])
center_df['launch_date'] = pd.to_datetime(center_df['launch_date'])

#center_df['competitor_entry_date'] = pd.to_datetime(center_df['competitor_entry_date'])
center_df['gx_entry_date'] = pd.to_datetime(center_df['gx_entry_date'])

# fillna to gx_entry_date with mode of gx entry date -- Hang added
month_add = ((center_df.gx_entry_date.dt.year - center_df.launch_date.dt.year)*12 + (center_df.gx_entry_date.dt.month - center_df.launch_date.dt.month)).mode()
center_df.loc[pd.isna(center_df.gx_entry_date),'gx_entry_date'] = center_df.loc[pd.isna(center_df.gx_entry_date)].launch_date + pd.offsets.DateOffset(months = int(month_add))
center_df.head()

(147397, 10)
(80358, 11)
(878,)


,index_x,country,cluster,cluster_id,date,stage_name,stage_name_lag_1,volume,business_unit,ther_area_fact,prevalence,gx_entry_date,index_y,launch_date,start_date
0,1860,country_0,brand_43,ID_18,2012-01-01,Growth,No_Stage,2.795290e+10,TWO,6,-1.0,2016-08-01,17,2008-08-01,2012-01-01
1,1861,country_0,brand_43,ID_18,2012-02-01,Growth,No_Stage,3.315936e+10,TWO,6,-1.0,2016-08-01,17,2008-08-01,2012-01-01
2,1862,country_0,brand_43,ID_18,2012-03-01,Growth,No_Stage,3.579896e+10,TWO,6,-1.0,2016-08-01,17,2008-08-01,2012-01-01
3,1863,country_0,brand_43,ID_18,2012-04-01,Growth,No_Stage,3.803040e+10,TWO,6,-1.0,2016-08-01,17,2008-08-01,2012-01-01
4,1864,country_0,brand_43,ID_18,2012-05-01,Growth,No_Stage,3.277213e+10,TWO,6,-1.0,2016-08-01,17,2008-08-01,2012-01-01


In [ ]:
temp = transition_point(center_df)
temp.query("transition_point == 1 & stage_name != 'Maturity'").head()

,index_x,country,cluster,cluster_id,date,stage_name,stage_name_lag_1,volume,business_unit,ther_area_fact,prevalence,gx_entry_date,index_y,launch_date,start_date,transition_point


In [ ]:
#center_df = comp_num(center_df)
#center_df = first_comp_month(center_df)
#center_df = center_df.drop(['competitor_entry_date'], axis=1).drop_duplicates()

#launch_months(center_.0df)
#time_since_no_exclusivity(center_df)
#transition_point(center_df)

#center_df = is_exclusive(center_df)
#center_df = last_stage_days(center_df)


#print(center_df[center_df['date']>'2019-03-01'])
#center_df = feature_hang(center_df)
#print(center_df[center_df['date']>'2019-03-01'])
#center_df['prevalence'] = center_df.prevalence_pct / center_df.population

#print(center_df.columns)
#print(center_df.shape)

final_df = senqi_transform(center_df)
final_df = feature_hang(final_df)
final_df = feature_lindsey(final_df)
final_df = categoricalize(final_df)
#final_df['prevalence'] = final_df.prevalence_pct / final_df.population
final_df = final_df.drop(["index_x","index_y"],axis=1)

In [ ]:
final_df.head()

,country,cluster,cluster_id,date,stage_name,stage_name_lag_1,volume,business_unit,ther_area_fact,prevalence,...,3,4,5,6,7,8,9,10,11,12
0,country_0,brand_43,ID_18,2012-01-01,Growth,No_Stage,2.795290e+10,TWO,6,-1.0,...,0,0,0,0,0,0,0,0,0,0
1,country_0,brand_43,ID_18,2012-02-01,Growth,No_Stage,3.315936e+10,TWO,6,-1.0,...,0,0,0,0,0,0,0,0,0,0
2,country_0,brand_43,ID_18,2012-03-01,Growth,No_Stage,3.579896e+10,TWO,6,-1.0,...,1,0,0,0,0,0,0,0,0,0
3,country_0,brand_43,ID_18,2012-04-01,Growth,No_Stage,3.803040e+10,TWO,6,-1.0,...,0,1,0,0,0,0,0,0,0,0
4,country_0,brand_43,ID_18,2012-05-01,Growth,No_Stage,3.277213e+10,TWO,6,-1.0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
final_df.to_csv("final_df.csv",index=False)

In [ ]:
final_df_v2 = feature_hang_addition(final_df)
final_df_v2.to_csv("final_df_v2.csv",index=False)

In [ ]:
final_df_v3 = feature_lindsey_addition(final_df_v2)
final_df_v3.to_csv("final_df_v3.csv",index=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
final_df_v3.head()

,country,cluster,cluster_id,date,stage_name,stage_name_lag_1,volume,business_unit,ther_area_fact,prevalence,...,vol_std_alltime,vol_mean_before_2016,vol_std_before_2016,vol_mean_before_4m,vol_std_before_4m,vol_mean_before_1y,vol_std_before_1y,growth_date,month_since_growth,volume_lag_1m_pctg
0,country_0,brand_43,ID_18,2012-01-01,Growth,No_Stage,2.795290e+10,TWO,6,-29.393769,...,3.596694e+10,6.477512e+10,2.148358e+10,9.246055e+10,1.734120e+10,8.693251e+10,1.300625e+10,2012-01-01,0,NaN
1,country_0,brand_43,ID_18,2012-02-01,Growth,No_Stage,3.315936e+10,TWO,6,-29.393769,...,3.596694e+10,6.477512e+10,2.148358e+10,9.246055e+10,1.734120e+10,8.693251e+10,1.300625e+10,2012-01-01,1,0.281885
2,country_0,brand_43,ID_18,2012-03-01,Growth,No_Stage,3.579896e+10,TWO,6,-29.393769,...,3.596694e+10,6.477512e+10,2.148358e+10,9.246055e+10,1.734120e+10,8.693251e+10,1.300625e+10,2012-01-01,2,0.180428
3,country_0,brand_43,ID_18,2012-04-01,Growth,No_Stage,3.803040e+10,TWO,6,-29.393769,...,3.596694e+10,6.477512e+10,2.148358e+10,9.246055e+10,1.734120e+10,8.693251e+10,1.300625e+10,2012-01-01,3,0.182908
4,country_0,brand_43,ID_18,2012-05-01,Growth,No_Stage,3.277213e+10,TWO,6,-29.393769,...,3.596694e+10,6.477512e+10,2.148358e+10,9.246055e+10,1.734120e+10,8.693251e+10,1.300625e+10,2012-01-01,4,-0.387267


In [ ]:
#Generate plots

#Target: trainsition point, regulation designation, indication, competitor number



#fig,ax = plt.subplots(2,2,figsize = (15,10))
#sns.countplot(data=final_df,x="transition_point",ax=ax[0,0]).set(title='Distribution of exlusivity')
#sns.countplot(data=final_df,x="comp_num",ax=ax[0,1]).set(title='Distribution of competitor numbers')
#sns.countplot(data=final_df,x="indication_num",ax=ax[1,0]).set(title='Distribution of indication numbers')
#sns.countplot(data=final_df,x="regulatory_designations_hasany",ax=ax[1,1]).set(title='Distribution of regulatory designations')

In [ ]:
# final_df[final_df.prevalence_pct <= 0]['prevalence_pct'] = np.nan
# final_df[['prevalence_pct']].value_counts()

## Feature for Phase 2

* month_since_transition (Hang)
* various volume-related features (growth rate, etc.) (Hang)

In [ ]:
def month_since_GtoM_trans(df):
  test = df.copy()
  test['rn'] = test.query(" stage_name == 'Maturity' & stage_name_lag_1 == 'Growth' ").groupby(['cluster_id','stage_name','stage_name_lag_1'],as_index=False)['date'].rank(method = 'first')
  #test['rn'] = test.rn.fillna(0)
  test = test.rename(columns = {'rn':'month_since_GtoM_trans'})
  return test

In [ ]:
def growth_rate(df):
  # vol_standard
  test = df.copy()
  test = test.assign( vol_standard = (test['volume'] - test['vol_mean_alltime']) / test['vol_std_alltime'] )
  # growth_change_rate
  a = test.query(" stage_name == 'Growth' & stage_name_lag_1 == 'No_Stage' ").groupby('cluster_id')['volume'].min() 
  b = test.query(" stage_name == 'Growth' & stage_name_lag_1 == 'No_Stage' ").groupby('cluster_id')['volume'].max() 
  output_right = ((b - a) / a).reset_index().rename(columns = {"volume":"growth_change_rate"})
  test = test.merge(output_right, how = 'left', on = 'cluster_id')
  # adding later (growth period mean and std)
  return test

In [ ]:
df_vol = month_since_GtoM_trans(final_df_v3)
df_vol = growth_rate(df_vol)

In [ ]:
p1 = df_vol.loc[0:df_vol.shape[0]/2]
p2 = df_vol.loc[df_vol.shape[0]/2 + 1 : df_vol.shape[0] ]
p1.to_csv("df_vol1.csv",index=False)
p2.to_csv("df_vol2.csv",index=False)
print(p1.shape)
print(p2.shape)
print(p1.shape[0] + p2.shape[0], df_vol.shape[0])

(40180, 51)
(40178, 51)
80358 80358


In [ ]:
#print(final_df_v3.__dict__.keys())

In [ ]:
print(df_vol.shape)
print(final_df.shape)
print(final_df_v2.shape)
print(final_df_v3.shape)

(80358, 51)
(80358, 37)
(80358, 47)
(80358, 48)


## Quality Check

Check if our features are created correctly. Choose one cluster  and manually find the ground truths. (ID_3360 right here)



In [ ]:
final_df.columns

In [ ]:
final_df.head(1).T

In [ ]:
temp_df = center_df
# final_transformation(temp_df)

temp_df = senqi_transform(temp_df)
temp_df = feature_hang(temp_df)

print(temp_df.shape)
temp_df.columns

ID_22:
- no gx entry
- 1 competitor entry date: 2006-03-01
- Transition point: 2014-02-01 (checked)
- Never lose exclusivity (checked)
- Launch months 2012-01-01 (checked)
- augment transition points (checked)
- country: country_0 (no given population information) (checked)
- reg_deg: 1.0 (checked)
- no indication (checked)



In [ ]:
board[board.cluster_id == "ID_22"].head()

In [ ]:
#cx_df[cx_df.cluster_id == "ID_22"]
#gx_df[gx_df.cluster_id == "ID_22"]
#country_pop[country_pop.country == 'country_0']
#reg_deg[reg_deg.cluster_id == "ID_22"]
# indication_df[indication_df.cluster_id == "ID_22"]

In [ ]:
#display(temp_df.loc[(temp_df.cluster_id == "ID_22"),['launch_date','date','launch_months']])     ## check launch months: how many months since the launch date.
display(temp_df.loc[(temp_df.cluster_id == "ID_22"),['time_since_no_exclusivity']])    ## time_since_lost_exclusivity:
#display(temp_df.loc[(temp_df.cluster_id == "ID_22") &( temp_df.transition_point == 1)]) ## check transition points and augment points 
#display(temp_df.loc[(temp_df.cluster_id == "ID_22"),['population']])  ##check population
#display(temp_df.loc[(temp_df.cluster_id == "ID_22"),['regulatory_designations_hasany']])  ##check 'regulatory_designations_hasany'
#display(temp_df.loc[(temp_df.cluster_id == "ID_22"),['indication_num']])  ##check indication_num



In [ ]:
temp_df[temp_df.cluster_id == "ID_47"].head()

ID_3360:
- GX date :2022-03-01 (time_since_no_exclusivity checked)
- 1 competitor entry date: 2020-11-01
- no Transition point (checked)
- lose exclusivity (checked)
- Launch months since 2014-07-01 (checked)
- no augment transition points (checked)
- country_34 (no population information) (checked)
- no reg_deg (checked)
- no indication(checked)








In [ ]:
temp_df[temp_df.cluster_id == "ID_3360"].head()

In [ ]:
# gx_df[gx_df.cluster_id == "ID_3360"]
# cx_df[cx_df.cluster_id == "ID_3360"]
#country_pop[country_pop.country == 'country_34']
#reg_deg[reg_deg.cluster_id == "ID_3360"]
#indication_df[indication_df.cluster_id == "ID_3360"]

In [ ]:
#display(temp_df.loc[(temp_df.cluster_id == "ID_3360"),['launch_date','date','launch_months']])     ## check launch months: how many months since the launch date.
#display(temp_df.loc[(temp_df.cluster_id == "ID_3360"),['date','time_since_no_exclusivity']])    ## time_since_lost_exclusivity:
#display(temp_df.loc[(temp_df.cluster_id == "ID_3360") &( temp_df.transition_point == 1)]) ## check transition points and augment points 
#display(temp_df.loc[(temp_df.cluster_id == "ID_3360"),['regulatory_designations_hasany']])  ##check 'regulatory_designations_hasany'
#display(temp_df.loc[(temp_df.cluster_id == "ID_3360"),['indication_num']])  ##check indication_num


**id 22 and id 3360 neither has information about population and indications, 
we choose id 755 to check these two features:**

id 755
- population: country_8 (checked)
- indication date: 2012-04-01, 2018-04-01, 2018-11-01, 2018-11-01 (checked)

In [ ]:
temp_df[temp_df.cluster_id == "ID_755"].head()

In [ ]:
#country_pop[country_pop.country == 'country_8'].sort_values(by = ['year'])
#indication_df[(indication_df.cluster_id == "ID_755") & (indication_df.indication_entry == 1)]

In [ ]:
#display(temp_df.loc[(temp_df.cluster_id == "ID_755"),['year','population']])  ##check population
# display(temp_df.loc[(temp_df.cluster_id == "ID_755") & (temp_df.indication_num == 1),['date','indication_num']])  ###check indication_nums
# display(temp_df.loc[(temp_df.cluster_id == "ID_755") & (temp_df.indication_num == 2),['date','indication_num']])
# display(temp_df.loc[(temp_df.cluster_id == "ID_755") & (temp_df.indication_num == 3),['date','indication_num']])

In [ ]:
# qc = center_df[center_df.cluster_id == "ID_22"]

In [ ]:
# center_df = feature_lindsey(center_df)
# center_df.head()

In [ ]:
gx_df[gx_df.cluster_id == 'ID_45']

In [ ]:
cx_df[cx_df.cluster_id == 'ID_1405']

In [ ]:
#center_df[center_df.cluster_id == 'ID_1405'][['comp_num', 'first_comp_month', 'is_exclusive', 'last_stage_days']]

In [ ]:
#center_df[center_df.cluster_id == 'ID_45'][['date', 'comp_num', 'first_comp_month', 'is_exclusive', 'last_stage_days']]

In [ ]:
final_df.is_exclusive.value_counts().plot(kind='barh')

In [ ]:
final_df

Hang Working Below

In [ ]:
board.query("cluster_id == 'ID_193'").stage_name.value_counts()

In [ ]:
cols = ['cluster_id','date','stage_name', 'stage_name_lag_1', 'volume']

In [ ]:
board.query("cluster_id == 'ID_193'").loc[:,cols]

In [ ]:
board['date'] = pd.to_datetime(board.date)
sns.lineplot(x='date',y='volume',hue='stage_name', data=board.query("cluster_id == 'ID_193'"))